### Try different similarity measures, using various representations and metrics
#### Create functions and class attributes when needed

In [87]:
%load_ext autoreload
%autoreload 2
from gensim.parsing.preprocessing import *
import io
from betterdictionary import BetterDictionary
from representation import Represent

elections = io.open('../data/Election2008Paragraphes.txt',encoding = "ISO-8859-1")
electionlines  =elections.readlines()

CUSTOM_FILTERS = [lambda x: x.lower(),  strip_punctuation, strip_multiple_whitespaces, strip_numeric,remove_stopwords, strip_short]

texts = [preprocess_string(line, filters=CUSTOM_FILTERS) for line in electionlines]
texts =filter(None,texts)

td = BetterDictionary(texts)

r = Represent(td, texts)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
r.build_word2vec(size=100, sg=1, window=30)
r.build_word2vec(size=300, sg=1, window=30)
r.build_word2vec(size=300, sg=1, window=100)


In [85]:
r.build_many_w2v(sizes=[100,300], windows =[10,100])

In [10]:
print r.w2v200skipgram30.most_similar('warming')

[(u'global', 0.8782487511634827), (u'atmosphere', 0.8509193658828735), (u'climate', 0.8426602482795715), (u'emitting', 0.8343095779418945), (u'greenhouse', 0.8279435634613037), (u'fossil', 0.8262056112289429), (u'kyoto', 0.8251897096633911), (u'caps', 0.8248090744018555), (u'droughts', 0.8219916820526123), (u'emission', 0.819460391998291)]


In [89]:
import pandas as pd
import qgrid
pd.option_context('display.max_rows', None, 'display.max_columns', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

similarities= r.n_most_similar('warming', n=30)

display(pd.DataFrame(similarities))


direct                                                                                                                                                       indirect                                                    
   sim_association sim_correlation       sim_dice     sim_gmean sim_inclusion    sim_jaccard sim_joint_prob  sim_log_cond      sim_npmi     sim_pmi     sim_zscore sim_cosine_ind w2v100skipgram30 w2v300skipgram100 w2v300skipgram30
0          warming         warming        warming       warming       pollute        warming        warming       packing       warming     warming        warming        warming           global            global           global
1           global          global         global        global        censor         global         global      megawatt        global   epidemics         global         global       atmosphere        atmosphere       atmosphere
2        epidemics       epidemics      emissions     epidemics        costal      emissions         energy     viewpoint     epidemics      forest      epidemics      emissions         emitting          emitting          climate
3        emissions       emissions      pollution     emissions   collaborate      pollution          world      montreal        forest       alarm      emissions         energy          climate             kyoto         emitting
4        pollution       pollution         reduce     pollution        hansen         reduce            new         layer         alarm     mercury      pollution      pollution       greenhouse           climate       greenhouse
5            alarm           alarm        climate         alarm     imperiled        climate            oil         yukon        spruce  whitehorse          alarm      epidemics             caps            fossil            kyoto
6           energy          forest         energy        energy     fostering         energy      emissions    whitehorse        costal     swallow         energy        climate           fossil              caps             caps
7           forest            halt      epidemics        forest       furious      epidemics         reduce        spruce         twain  forbidding           halt  environmental         emission       catastrophe         emission
8             halt          energy         carbon          halt     bankrolls         carbon        america       cognize       natives     weakest         forest          clean            kyoto          emission           fossil
9           drives          drives          clean        drives         flash          clean           need         passe      textiles    releases         drives         carbon      catastrophe        greenhouse         droughts
10        droughts         sounded          india      droughts        forest          india          clean       lindsey       lindsey      ottawa         reduce         reduce         droughts          droughts      catastrophe
11       designing       designing        science     designing    consultant        science        foreign       insects       pollute    shutdown           curb     developing        addiction              atom        addiction
12         forests    inauguration  environmental       forests     censoring  environmental      president      shutdown         flash     insects          polar            oil         protocol              acid             acid
13         sounded         forests            cap       sounded    proverbial            cap         states        ottawa    whitehorse     natives           caps         planet           gasses          protocol        emissions
14    inauguration        droughts         planet  inauguration     reprocess         planet         people     reprocess         yukon     lindsey          india   technologies             atom         addiction         protocol
15           polar            curb            oil         polar        client            oil          

In [183]:
from collections import Counter
df=pd.DataFrame(similarities)
counts =Counter(df.values.flatten())
import matplotlib.pyplot as plt  
from matplotlib import colors
countsd = {}
for x,v in similarities.items():
    countsd[x]= [counts[vx] for vx in v]
    
A =pd.DataFrame(countsd)
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
def b_g(s, cmap='PuBu', low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    rng = a.max() - a.min()
    
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

colored =pd.DataFrame(similarities).style.apply(b_g,cmap= sns.light_palette("green", reverse=True,as_cmap=True))
display(colored)

@interact
def show_words_by_count(count = 20):
    dff =pd.DataFrame(similarities).style.apply(b_g,cmap= sns.light_palette("green", reverse=True,as_cmap=True))
    dff.where(A > count)
    return dff

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0yMCwgZGVzY3JpcHRpb249dSdjb3VudCcsIG1heD02MCwgbWluPS0yMCksIE91dHB1dCgpKSwgX2RvbV9jbGFzc2VzPSh1J3figKY=


In [181]:
d =A.where(A >10)
d.fillna('')

direct                                                                                                                            indirect                                                    
   sim_association sim_correlation sim_dice sim_gmean sim_inclusion sim_jaccard sim_joint_prob sim_log_cond sim_npmi sim_pmi sim_zscore sim_cosine_ind w2v100skipgram30 w2v300skipgram100 w2v300skipgram30
0                                                                                                                                                                    12                12               12
1               12              12       12        12                        12             12                    12      12         12             12                                                    
2               12              12       11        12                        11                                   12                 12             11                                                    
3               11              11                 11                                                                                11                                                                   
4                                                                                                                                                                                                         
5                                                                                                                                                   12                                                    
6                                                                                           11                                                                                                            
7                                        12                                  12                                                                                                                           
8                                                                                                                                                                                                         
9                                                                                                                                                                                                         
10                                                                                                                                                                                                        
11                                                                                                                                                                                                        
12                                                                                                                                                                                                        
13                                                                                                                                                                                                      11
14                                                                                                                                                                                                        
15                                                                                                                                                                                                        
16                                                                                                                                                                                                        
17                                                                                                                                                         

In [136]:
print countsd

{('direct', 'sim_joint_prob'): [10, 12, 8, 2, 2, 7, 11, 8, 1, 1, 7, 4, 1, 1, 1, 1, 10, 1, 1, 1, 1, 4, 1, 1, 5, 8, 1, 1, 1, 1], ('direct', 'sim_pmi'): [10, 12, 7, 7, 1, 3, 1, 2, 2, 3, 3, 3, 4, 3, 4, 3, 1, 2, 3, 3, 2, 2, 1, 3, 1, 2, 3, 3, 1, 2], ('indirect', 'w2v300skipgram100'): [12, 3, 3, 3, 10, 3, 7, 3, 3, 6, 7, 3, 4, 3, 4, 3, 3, 6, 11, 7, 3, 12, 3, 3, 6, 2, 3, 2, 1, 3], ('direct', 'sim_log_cond'): [1, 2, 1, 2, 3, 3, 3, 3, 1, 2, 4, 4, 3, 3, 2, 2, 1, 1, 2, 2, 3, 2, 3, 3, 3, 1, 1, 3, 2, 1], ('direct', 'sim_npmi'): [10, 12, 12, 7, 7, 3, 2, 1, 3, 2, 4, 3, 2, 3, 3, 1, 3, 3, 3, 4, 1, 3, 3, 3, 3, 3, 2, 2, 2, 2], ('direct', 'sim_zscore'): [10, 12, 12, 11, 8, 7, 8, 7, 7, 4, 8, 5, 7, 7, 6, 10, 7, 6, 5, 4, 4, 7, 4, 4, 7, 3, 4, 4, 4, 4], ('direct', 'sim_dice'): [10, 12, 11, 8, 8, 10, 8, 12, 5, 7, 6, 4, 4, 4, 3, 7, 3, 2, 7, 6, 2, 3, 6, 2, 3, 3, 3, 4, 2, 2], ('direct', 'sim_jaccard'): [10, 12, 11, 8, 8, 10, 8, 12, 5, 7, 6, 4, 4, 4, 3, 7, 3, 2, 7, 6, 2, 3, 6, 2, 3, 3, 3, 4, 2, 2], ('direct', 'sim_as

In [38]:
%%time
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

matr = r.cooc
print type(matr)
print r.dictionary[384]
cos = cosine_distances(matr[384], matr)
#cos =cdist(matr, matr, metric='cosine')
print cos

<class 'scipy.sparse.csc.csc_matrix'>
states
[[ 0.90944549  0.47728163  0.54036017 ...,  0.96199494  0.95982807
   0.91006839]]
Wall time: 251 ms


In [39]:
import numpy as np
sorted =np.argsort(cos)
print sorted[0,:]
for m in sorted[0,:30]:
    print r.dictionary[m]
#print [r.dictionary[m] for m in sorted[0,:10]]

[ 384  386  382 ..., 8592 9061 9217]
states
united
president
countries
presidency
america
lead
blue
world
waiting
running
common
nations
committed
secure
people
build
cooperation
partner
right
american
advance
israel
different
europe
red
senate
security
time
ready


In [229]:
from similarity import *
def aa(a,b):
    return a+b

dd = r.cooc.todense()
#i want to be able to say jaccard() over an entire matrix?
#would be like function_jaccard(dfs,dfs,)
#otherwise [for for e in denseco]
occs =[r.occurrences[i] for i in xrange(len(r.occurrences))]
r.cooc.setdiag(occs)
dictd = r.occurrences
cooc_dict =r.dictionary.cooc_dict
print len(dictd), len(cooc_dict)

15663 15663


In [208]:
%%time
for e in xrange(1000):
    for j in xrange(1000):
        sim_jaccard(dd[e,e], dd[j,j],dd[e,j])

Wall time: 3.08 s


In [235]:
%%time 
jac = [sim_jaccard(dd[e,e], dd[j,j],dd[e,j]) for e in xrange(1000) for j in xrange(1000)]

Wall time: 3.04 s


In [256]:
%%time
jac = [sim_jaccard(dictd[e], dictd[j],cooc_dict[max(e,j)][min(e,j)]) for e in xrange(5000) for j in xrange(5000)]

Wall time: 17.3 s


In [238]:
print len(jac)

1000000


In [234]:
%%time
jac = [sim_jaccard(dictd[e], dictd[j],dd[e,j]) for e in xrange(1000) for j in xrange(1000)]

Wall time: 1.57 s


In [313]:
print sim_pmi.func_code.co_argcount

5


In [10]:
import numpy as np
from similarity import *
ujac = np.frompyfunc(sim_jaccard,3,1)


In [ ]:
from itertools import combinations
woccs= [r.occurrences[1]]* len(r.dictionary)

ujac(woccs, r.occs,r.coocdense[1] )

In [16]:
a = np.arange(24).reshape(2,3,4)
print a

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


In [286]:
occso= occs*len(occs)

In [287]:
cocso =dd.flatten()
print len(cocso)

1


In [288]:
print cocso.shape

(1L, 245329569L)


In [289]:
%%time
uj = ujac(occso, occso, cocso)

Wall time: 59.9 s


In [255]:
print uj.shape

(25000000L,)


In [291]:
%%time
ujfirst = ujac(occs, occs, dd[0])

Wall time: 16 ms


In [15]:
print np.argsort(ujfirst)[0,-10:]

NameError: name 'np' is not defined

In [285]:
dff =dd.flatten()[0,:100]

print dff

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [290]:
print dd[0]

[[1 1 1 ..., 0 0 0]]


In [121]:
%%time
print vectors.cosine_matrix()

[[ 1.          0.11416432  0.12891311 ...,  0.          0.          0.        ]
 [ 0.11416432  1.          0.91075545 ...,  0.06794365  0.08296525
   0.13382715]
 [ 0.12891311  0.91075545  1.         ...,  0.0591066   0.0901368
   0.11819915]
 ..., 
 [ 0.          0.06794365  0.0591066  ...,  1.          0.          0.        ]
 [ 0.          0.08296525  0.0901368  ...,  0.          1.          0.        ]
 [ 0.          0.13382715  0.11819915 ...,  0.          0.          1.        ]]
Wall time: 20.6 s


In [125]:
%%time
af =vectors.cooc 
aft =vectors.cooc.T
np.dot(af,aft)

Wall time: 17.4 s


In [24]:
occ = np.array((td.dfs.values()))
print occ
print td.cooc_dict.items()
empty = np.zeros((len(occ),len(occ)))
print empty
df =pd.DataFrame(td.cooc_dict).fillna(0)
display(df)
#display(df.T)

[3 2 2 1 1]
[(0, defaultdict(<type 'int'>, {1: 2, 2: 2, 3: 1, 4: 1})), (1, defaultdict(<type 'int'>, {2: 1, 4: 1})), (2, defaultdict(<type 'int'>, {3: 1, 4: 1}))]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


,0,1,2
1,2,0.0,0.0
2,2,1.0,0.0
3,1,0.0,1.0
4,1,1.0,1.0


,1,2,3,4
0,2.0,2.0,1.0,1.0
1,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,1.0


In [37]:
# Find the combination of both indices
full_index = df.index.union(df.columns)

In [38]:
# Resize the DataFrame to include all the rows and columns
all_data = df.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1)

In [39]:
# Update any values we have from the transpose 
all_data.update(all_data.T)

In [41]:
# Fill the missing entries
result = all_data.fillna(0.0)
display(result)

,0,1,2,3,4
0,0.0,2.0,2.0,1.0,1.0
1,2.0,0.0,1.0,0.0,1.0
2,2.0,0.0,0.0,1.0,1.0
3,1.0,0.0,1.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0


In [41]:
rows =np.array(td.cooc_dict.keys())
colums = np.array([k.keys() for k in td.cooc_dict.values()])
print rows
print colums.flatten()
print td.cooc_dict.items()
print td.keys()

#co_occurrences[rows, colums.flatten()]

[0 1 2]
[[1, 2, 3] [2, 4] [3, 4]]
[(0, defaultdict(<type 'int'>, {1: 1, 2: 1, 3: 1})), (1, defaultdict(<type 'int'>, {2: 1, 4: 1})), (2, defaultdict(<type 'int'>, {3: 1, 4: 1}))]
[0, 2, 1, 3, 4]


In [43]:
a = np.array([[4,1,2], [3,4]])
a.flatten()

array([[4, 1, 2], [3, 4]], dtype=object)

In [54]:
def test(a,b):
    print a,b
    
l = [1,2,3]
print test(*l[:2])

 1 2
None
